<a href="https://colab.research.google.com/github/oguzhankoc55/Text-Smilarity-NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metin benzerligi 
Nlp dersinde verilen ara dönem projesi yerine geçen proje için oluşturulan belge.Bu projede metin benzerligi yöntemlerini kullanarak elimizdeki veriler ile hakem ataması denemeleri yaptık.

## Verilerin Yüklenmesi
Veriler drive üzerinde bulunan adresleri belirlenir ve bu adreslerdeki veriler çekilir.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os

In [ ]:
def make_data(path):
  veriler = []
  pdfler = os.listdir(path)
  for pdf in pdfler:
    try:
        veri= {
            "isim":"",
        }
        veri.update({"isim" : pdf})
        dosya = open(path+pdf,"r")
        for metin in dosya:
            veri.update({"metin":metin})
        
        dosya.close()
        veriler.append(veri)
    except:
        print( "hata")
  return veriler

In [ ]:
def refactor(path):
  results = []
  dosya = open(path,"r")
  for metin in dosya:
    veri= {
            "isim":"",
        }
    result = metin.replace("\n","").split("\t")
    _List = []
    for i in range(len(result)):
      if(i == 0 ):
        veri.update({"isim" : result[i]})
      else:
        _List.append({str(i):result[i]})
    veri.update({"revievers":_List})  
    results.append(veri)

  return results

In [ ]:
#adding datasets
path_manu= "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/manuscripts/"
path_revi= "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/reviewers/"
path_ground = "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/groundturth.txt"

manuscripts_veriler = make_data(path_manu)
revievers_veriler = make_data(path_revi)
groundturth_veriler = refactor(path_ground)

## Preprocessing

### noktalama işaretleri ve sayıların atılması

In [ ]:
def To_lower(veriler):
  for i in range(len(veriler)):
    veriler[i]["metin"] = veriler[i]["metin"].lower()
    veriler[i]["metin"] = veriler[i]["metin"].replace("\n"," ").replace("\t"," ")
    veriler[i]["metin"] = re.sub(r'[^a-zA-Z\s]', ' ', veriler[i]["metin"])
  return veriler

In [ ]:
manuscripts_veriler=To_lower(manuscripts_veriler)
revievers_veriler=To_lower(revievers_veriler)

### stopwordslerin çıkarılması

In [ ]:
nltk.download('stopwords') 
stop_words =  set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

def del_stopWords(veriler):
  for i in range(len(veriler)):
    veriler[i]["metin"]  = " ".join([c for c in veriler[i]["metin"].split() 
    if c not in stop_words if len(c)>1])
  return veriler

In [ ]:
manuscripts_veriler = del_stopWords(manuscripts_veriler)
revievers_veriler= del_stopWords(revievers_veriler)

In [ ]:
stemmer = nltk.porter.PorterStemmer()

In [ ]:
metin = ' '.join([stemmer.stem(word) for word in manuscripts_veriler[1]["metin"].split()])
metin

'higher spin black hole soft hair construct new set boundari condit higher spin graviti inspir recent soft heisenberg hair propos gener rel onthre dimension anti de sitter asymptot symmetri algebra consist aset affin hat current algebra associ canon chargesgener higher spin soft hair focu first spin case thenextend main result spin mani resembl spin result gener asymptot algebra natur emerg fromcomposit oper hat charg twist sugawaraconstruct boundari condit ensur regular euclideansolut space independ valu charg solut wecal higher spin black flower stationari necessarilyspher symmetr final deriv entropi higher spin blackflow find branch continu connect btzblack hole depend affin pure gravit zero mode use map algebra current recov well known express forhigh spin entropi also address higher spin black flower metricform achiev full consist previou result'

In [ ]:
manuscripts_veriler[1]["metin"]

'higher spin black holes soft hair construct new set boundary conditions higher spin gravity inspired recent soft heisenberg hair proposal general relativity onthree dimensional anti de sitter asymptotic symmetry algebra consists aset affine hat current algebras associated canonical chargesgenerate higher spin soft hair focus first spin case thenextend main results spin many resemble spin results generators asymptotic algebra naturally emerge fromcomposite operators hat charges twisted sugawaraconstruction boundary conditions ensure regularity euclideansolutions space independently values charges solutions wecall higher spin black flowers stationary necessarilyspherically symmetric finally derive entropy higher spin blackflowers find branch continuously connected btzblack hole depends affine purely gravitational zero modes using map algebra currents recover well known expressions forhigher spin entropy also address higher spin black flowers metricformalism achieve full consistency prev